# Emerging Technologies
---

## Introduction

--- 

In [1]:
import numpy as np
import itertools

## Problem 1: Generating Random Boolean Functions
>The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations). Write a Python function random_constant_balanced that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

In [2]:
def random_constant_balanced():
    """
    Returns a randomly chosen function (constant or balanced) 
    taking four Boolean arguments as input.
    
    The function represents a 4-bit black-box function (Oracle). Constant functions 
    return a single value for all 16 inputs. Balanced functions 
    return True for 8 inputs and False for the other 8.

    Returns:
        function: A lambda function taking four Boolean arguments (a, b, c, d) 
                  representing the 4-bit input to the Oracle.
    """
    n = 4
    n_inputs = 2 ** n 

    # Decide randomly between constant and balanced
    # Constant function will output all True or all False
    if np.random.randint(0, 2):
        val = bool(np.random.randint(0, 2)) # Randomly choose True or False
        outputs = np.full(n_inputs, val, dtype=bool) # np.full creates an array filled with val (True or False)
        func_type = "Constant"
    # Balanced function will output half True and half False
    else:
        outputs = np.array([False] * (n_inputs // 2) + [True] * (n_inputs // 2)) # Half False, half True
        np.random.shuffle(outputs) # Shuffle to randomize order
        func_type = "Balanced"

    # Map 4 Boolean arguments to an integer index using binary weighting (8,4,2,1)
    # this allows us to use the pre-calculated 'outputs' array as a lookup table
    # See https://www.w3schools.com/PYTHON/python_lambda.asp for lambda examples and description
    f = lambda a, b, c, d: outputs[8*int(a) + 4*int(b) + 2*int(c) + 1*int(d)]
    f.type = func_type
    return f

# Quick verification of 16 possible inputs for 4 Boolean variables
inputs = list(itertools.product([False, True], repeat=4))

for i in range(10):
    f = random_constant_balanced()
    
    results = [f(*inp) for inp in inputs]
    t_count = sum(results)
    f_count = 16 - t_count
    
    print(f"Function: {f.type} (True: {t_count}, False: {f_count})")

Function: Balanced (True: 8, False: 8)
Function: Constant (True: 16, False: 0)
Function: Constant (True: 16, False: 0)
Function: Constant (True: 0, False: 16)
Function: Balanced (True: 8, False: 8)
Function: Balanced (True: 8, False: 8)
Function: Balanced (True: 8, False: 8)
Function: Constant (True: 0, False: 16)
Function: Constant (True: 0, False: 16)
Function: Constant (True: 0, False: 16)


---
## Problem 2: Classical Testing for Function Type
>Deutsch's algorithm is designed to demonstrate a potential advantage of quantum computing over classical computation. To understand this advantage, we must first understand the classical cost of solving the underlying problem. Write a Python function determine_constant_balanced that takes as input a function f, as defined in Problem 1. The function should analyze f and return the string "constant" or "balanced" depending on whether the function is constant or balanced. Write a brief note on the efficiency of your solution. What is the maximum number of times you need to call f to be 100% certain whether it is constant or balanced?

---
## Problem 3: Quantum Oracles
>Deutsch's algorithm is the simplest example of a quantum algorithm using superposition to determine a global property of a function with a single evaluation. In the single-input case, there are four possible Boolean functions. Using Qiskit, create the appropriate quantum oracles for each of the possible single-Boolean-input functions used in Deutsch's algorithm. Demonstrate their use and explain how each oracle implements its corresponding function.

---
## Problem 4: Deutsch's Algorithm with Qiskit
>Use Qiskit to design a quantum circuit that solves Deutsch's problem for a function with a single Boolean input. Implement the necessary circuit and demonstrate its use with each of the quantum oracles from Problem 3. Describe how the interference pattern produced by the circuit allows you to determine whether the function is constant or balanced using only one query to the oracle.

---
## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
>The Deutsch–Jozsa algorithm generalizes Deutsch's approach to functions with several input bits. Use Qiskit to create a quantum circuit that can handle the four-bit functions generated in Problem 1. Explain how the classical function is encoded as a quantum oracle, and demonstrate the use of your circuit on both of the constant functions and any two balanced functions of your choosing. Show that the circuit correctly identifies the type of each function.

---
## Conclusion

---
# End